# Libraries

In [2]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from urllib.parse import urlencode
import time

# Driver

In [4]:
def setup_driver():
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')
    options.add_argument('--disable-gpu')
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
    return driver

# URL

In [6]:
def generate_url(query, filters=None, page=None):
    params = {'q': query}
    if filters:
        params['a'] = filters
    if page and page > 1:
        params['start'] = (page - 1) * 20
    return f"https://wuzzuf.net/search/jobs/?{urlencode(params)}"

In [7]:
def scrape_jobs(job_titles, max_pages=0):
    driver = setup_driver()
    wait = WebDriverWait(driver, 10)
    jobs_data = []
    try:
        for job_title in job_titles:
            print(f"Scraping jobs for: {job_title}")
            
            for page in range(1, max_pages + 1):
                url = generate_url(job_title, "navbg|spbg", page)
                print(f"Processing page {page}: {url}")
                
                driver.get(url)
                wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "div.css-1gatmva.e1v1l3u10")))
                
                job_cards = driver.find_elements(By.CSS_SELECTOR, "div.css-1gatmva.e1v1l3u10")
                
                if not job_cards:
                    print(f"No jobs found on page {page} for {job_title}")
                    break
                
                for card in job_cards:
                    try:
                        job_link = card.find_element(By.CSS_SELECTOR, "a.css-o171kl").get_attribute("href")
                        
                        job_id = job_link.split('/')[-1]

                        job_info = {
                            "ID": job_id,
                            "Title": card.find_element(By.CSS_SELECTOR, "a.css-o171kl").text,
                            "Company": card.find_element(By.CSS_SELECTOR, "a.css-17s97q8").text,
                            "Location": card.find_element(By.CSS_SELECTOR, "span.css-5wys0k").text,
                            "Time_Posted": card.find_element(By.CSS_SELECTOR, "div.css-d7j1kk").text,
                            "Job_Type": card.find_element(By.CSS_SELECTOR, "span.css-1ve4b75.eoyjyou0").text,
                            "Search_Query": job_title,
                            "Page": page,
                            "Job_Link": job_link}
                        
                        driver.execute_script("window.open('');")
                        driver.switch_to.window(driver.window_handles[1])
                        driver.get(job_link)
                        
                        try:
                            full_description = wait.until(
                                EC.presence_of_element_located((By.CSS_SELECTOR, "div.css-1uobp1k")))
                            job_info["Description"] = full_description.text
                            
                            try:
                                skills_elements = wait.until(
                                    EC.presence_of_all_elements_located((By.CSS_SELECTOR, "span.css-158icaa")))
                                job_info["Skills"] = ", ".join([skill.text for skill in skills_elements])
                            except:
                                job_info["Skills"] = "Not specified"
                            
                            try:
                                job_details = wait.until(
                                    EC.presence_of_all_elements_located((By.CSS_SELECTOR, "div.css-rcl8e5")))
                                
                                for detail in job_details:
                                    if "Experience Needed:" in detail.text:
                                        exp_text = detail.text
                                        job_info["Experience_Needed"] = exp_text.split("Experience Needed:")[1].strip()
                                        break
                                else:
                                    job_info["Experience_Needed"] = "Not specified"
                            except Exception as e:
                                print(f"Error extracting experience: {e}")
                                job_info["Experience_Needed"] = "Not specified"
                            
                        except Exception as e:
                            print(f"Error extracting details from job page: {e}")
                            job_info["Description"] = "Description not found"
                            job_info["Skills"] = "Not specified"
                            job_info["Experience_Needed"] = "Not specified"
                        
                        jobs_data.append(job_info)
                        
                        driver.close()
                        driver.switch_to.window(driver.window_handles[0])
                        
                    except Exception as e:
                        print(f"Error extracting job details: {e}")
                        if len(driver.window_handles) > 1:
                            driver.close()
                            driver.switch_to.window(driver.window_handles[0])
                        continue
                
                next_buttons = driver.find_elements(By.CSS_SELECTOR, "button.css-zye1os.ezfki8j0")
                if not any("Next" in btn.text for btn in next_buttons):
                    break
    finally:
        driver.quit()
    return pd.DataFrame(jobs_data)

# Jops you want to add

In [9]:
job_titles = ["Python","sowftware Engineer"]

In [10]:
jobs_df = scrape_jobs(job_titles, max_pages=2)

Scraping jobs for: Python
Processing page 1: https://wuzzuf.net/search/jobs/?q=Python&a=navbg%7Cspbg
Scraping jobs for: sowftware Engineer
Processing page 1: https://wuzzuf.net/search/jobs/?q=sowftware+Engineer&a=navbg%7Cspbg


In [11]:
jobs_df.head()

,ID,Title,Company,Location,Time_Posted,Job_Type,Search_Query,Page,Job_Link,Description,Skills,Experience_Needed
0,kx4QTsoTSbKX-Senior-Python-Developer-Softxpert...,Senior Python Developer,Softxpert Incorporation -,"Bolkly, Alexandria, Egypt","Softxpert Incorporation - Bolkly, Alexandria, ...",Full Time,Python,1,https://wuzzuf.net/jobs/p/kx4QTsoTSbKX-Senior-...,"Job Description:\nDevelop, test, and maintain ...","Business Development, IT/Software Development,...",More Than 3 Years
1,y9FsWhj1fMll-Python-Team-Lead---AI-Python-Engi...,Python Team Lead - AI & Python Engineering,Softec Technologies -,"Cairo, Egypt","Softec Technologies - Cairo, Egypt\n11 days ago",Full Time,Python,1,https://wuzzuf.net/jobs/p/y9FsWhj1fMll-Python-...,Position Overview:\nWe are seeking a highly sk...,"IT/Software Development, Quality, Engineering ...",More Than 6 Years
2,ayajJH3Afl3R-Python-Automation-Engineer-Luxoft...,Python Automation Engineer,Luxoft (A DXC Technology Company) -,"Cairo, Egypt","Luxoft (A DXC Technology Company) - Cairo, Egy...",Full Time,Python,1,https://wuzzuf.net/jobs/p/ayajJH3Afl3R-Python-...,Requirement analysis\nConcept development for ...,"IT/Software Development, Engineering - Telecom...",More Than 3 Years
3,kJNETgKZ7v9r-Senior-Python-Automation-Engineer...,Senior Python Automation Engineer,Luxoft (A DXC Technology Company) -,"Cairo, Egypt","Luxoft (A DXC Technology Company) - Cairo, Egy...",Full Time,Python,1,https://wuzzuf.net/jobs/p/kJNETgKZ7v9r-Senior-...,Requirement analysis\nConcept development for ...,"IT/Software Development, Engineering - Telecom...",More Than 5 Years
4,bwni7B3ow3qc-Python-Backend-Developer-%E2%80%9...,Python Backend Developer – AI & Chatbots,OTIC -,"New Cairo, Cairo, Egypt","OTIC - New Cairo, Cairo, Egypt\n1 month ago",Full Time,Python,1,https://wuzzuf.net/jobs/p/bwni7B3ow3qc-Python-...,Python Backend Developer\nAre you a skilled Py...,"IT/Software Development, Python Backend Develo...",2 To 4 Years


In [12]:
jobs_df.to_csv("wuzzuf_jobs3.csv",index=False)
print(f"Total jobs scraped:{len(jobs_df)}")

Total jobs scraped:30
